# 01a. Australian Open Tournament Info Scraper

Notebook will contain codes and functions for scraping the AO tournament page for a specific year.
Scraping functions will return datasets containing tournament and match information, as well as the URL for each match's match centre page. 

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import json
import itertools
import sys
sys._enablelegacywindowsfsencoding() #Deal with pandas problem with reading file with accents in file path i.e Alexis Sánchez, Victor Lindelöf 


headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'} 

In [32]:
from ast import literal_eval

In [ ]:
df_results_list = []
df_rounds_list = []

for day in np.arange(1,15):

    link = f'https://prod-scores-api.ausopen.com/year/2023/period/MD/day/{day}/results'
    # Get request and content from the given link and parse into HTML
    pageTree = requests.get(link, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser') 
    
    results_json = json.loads(str(pageSoup))
    df_results = pd.DataFrame(results_json['matches'])

    df_rounds_list.append(pd.DataFrame(results_json['rounds']))
    df_results_list.append(df_results)

In [2]:
# df_results = pd.concat(df_results_list).reset_index(drop=True)
# df_results.head()

In [33]:
#df_results.to_csv("data/AO_results-all_raw_2023.csv", index=False)
#df_rounds = pd.concat(df_rounds_list).drop_duplicates().reset_index(drop=True)
#pd.concat(df_rounds_list).drop_duplicates().reset_index(drop=True).to_csv("data/AO_rounds-uuid_2023.csv", index=False)
df_results = pd.read_csv("data/AO_results-all_raw_2023.csv", converters={"teams": literal_eval})
df_rounds = pd.read_csv("data/AO_rounds-uuid_2023.csv")

In [5]:
df_players_list = []
df_teams_list = []

for day in np.arange(1,15):

    link = f'https://prod-scores-api.ausopen.com/year/2023/period/MD/day/{day}/results'
    # Get request and content from the given link and parse into HTML
    pageTree = requests.get(link, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser') 
    
    results_json = json.loads(str(pageSoup))
    df_players_list.append(pd.DataFrame(results_json['players']))
    df_teams_list.append(pd.DataFrame(results_json['teams']))


In [22]:
df_teams = pd.concat(df_teams_list).drop_duplicates(subset=['uuid']).reset_index(drop=True)
df_teams.to_csv("data/AO_teams_2023.csv", index=False)
df_teams.to_json("data/AO_teams_2023.json")

In [23]:
df_players = pd.concat(df_players_list).drop_duplicates(subset=['full_name']).reset_index(drop=True)#.to_csv("data/AO_players_2023.csv", index=False)
df_players[['uuid', 'nid', 'player_id', 'tour_id', 'full_name']]

,uuid,nid,player_id,tour_id,full_name
0,3e34d8a0-003d-4fba-929f-74dc7058c52c,5764,ATPCG80,ATPCG80,Borna Coric
1,70ebe882-a3bd-410a-884e-1bc5e0144423,5781,ATPDA81,ATPDA81,Taro Daniel
2,659c0662-aec1-4ad3-b4b0-f484ab7b1525,5793,ATPE831,ATPE831,Kyle Edmund
3,a1cf153a-2309-462e-80c8-bc3553caad09,5794,ATPE873,ATPE873,Ernesto Escobedo
4,5e0139b2-be80-4349-b8ec-10c985272b5b,5800,ATPF724,ATPF724,Marton Fucsovics
...,...,...,...,...,...
588,12bff3fc-360b-4974-a504-c25eab3fbf70,238291,ITF800512726,ITF800512726,Shiori Funamizu
589,3dbc8dac-767b-4b41-b583-91417b3173bf,238296,ITF800232576,ITF800232576,Katharina Kruger
590,7354da53-2797-4f7e-b402-7eb92936e7a1,238336,ITF800470585,ITF800470585,Maria Florencia Moreno
591,dc148084-861f-4350-b5f9-1018e1bf6602,15561,ATPS694,ATPS694,Radek Stepanek


In [44]:
round_names = df_results.round_id.apply(lambda x : df_rounds[df_rounds.uuid == x].name.iloc[0])

In [ ]:
df_results.insert(17, "player2_name", p2_names)
df_results.insert(16, "player1_name", p1_names)
df_results.insert(14, "round", round_names)

In [ ]:
df_results.head(2)

,date,actual_start_time,match_centre_link,uuid,match_id,team_substituted_footnote,team_substituted,id,order,promoted,...,teams,player1_name,event_uuid,player2_name,court_id,session,session_order,restricted_start_time,restricted_start_time_timestamp,activity_order
0,2023-01-16,00:14,https://ausopen.com/match/2023-katerina-siniak...,222446,WS116,,False,222446,16,True,...,"[{'team_id': '226996', 'score': [{'set': 1, 'g...",Katerina Siniakova,4225c89e-7cfd-463f-a01d-4102ba2913f3,Coco Gauff,12,Day session,0,None,NaN,0
1,2023-01-16,01:50,https://ausopen.com/match/2023-yue-yuan-vs-mar...,222406,WS132,,False,222406,32,False,...,"[{'team_id': '227111', 'score': [{'set': 1, 'g...",Yue Yuan,4225c89e-7cfd-463f-a01d-4102ba2913f3,Maria Sakkari,12,Day session,0,None,NaN,1


In [43]:
p1_uuid = df_results.teams.apply(lambda x : df_teams[df_teams.uuid==x[0]['team_id']].players.iloc[0] )
p2_uuid = df_results.teams.apply(lambda x : df_teams[df_teams.uuid==x[1]['team_id']].players.iloc[0] )

In [45]:
df_results.insert(14, "round", round_names)

In [47]:
df_results.insert(17, "team2_player_uuid", p2_uuid)
df_results.insert(17, "team1_player_uuid", p1_uuid)

In [54]:
player1_names = df_results.team1_player_uuid.apply(lambda x : ", ".join([df_players[df_players.uuid==name].full_name.iloc[0] for name in x]) )
player2_names = df_results.team2_player_uuid.apply(lambda x : ", ".join([df_players[df_players.uuid==name].full_name.iloc[0] for name in x]) )

In [62]:
df_results.insert(list(df_results.columns).index("team2_player_uuid")+1, "player2_name", player2_names)
df_results.insert(list(df_results.columns).index("team1_player_uuid")+1, "player1_name", player1_names)

In [68]:
df_results

,date,actual_start_time,match_centre_link,uuid,match_id,team_substituted_footnote,team_substituted,id,order,promoted,...,player1_name,team2_player_uuid,player2_name,event_uuid,court_id,session,session_order,restricted_start_time,restricted_start_time_timestamp,activity_order
0,2023-01-16,00:14,https://ausopen.com/match/2023-katerina-siniak...,222446,WS116,NaN,False,222446,16,True,...,Katerina Siniakova,[2435e580-912e-4239-92c7-5778a17a7190],Coco Gauff,4225c89e-7cfd-463f-a01d-4102ba2913f3,12,Day session,0,NaN,NaN,0
1,2023-01-16,01:50,https://ausopen.com/match/2023-yue-yuan-vs-mar...,222406,WS132,NaN,False,222406,32,False,...,Yue Yuan,[d5ff642d-4e0f-4e07-bcc9-1cffa2fe0dee],Maria Sakkari,4225c89e-7cfd-463f-a01d-4102ba2913f3,12,Day session,0,NaN,NaN,1
2,2023-01-16,03:51,https://ausopen.com/match/2023-rafael-nadal-vs...,225666,MS101,NaN,False,225666,1,True,...,Rafael Nadal,[cd126eb3-67bf-4f1d-bd7c-c85a81b43afb],Jack Draper,675d28ec-b177-46b2-959c-595f3ca862a0,12,Day session,0,14:30,1.673840e+09,2
3,2023-01-16,08:34,https://ausopen.com/match/2023-iga-swiatek-vs-...,222471,WS101,NaN,False,222471,1,True,...,Iga Swiatek,[a386d06b-811c-486b-96c6-45d54f560ae3],Jule Niemeier,4225c89e-7cfd-463f-a01d-4102ba2913f3,12,Night session,1,NaN,NaN,0
4,2023-01-16,11:02,https://ausopen.com/match/2023-marcos-giron-vs...,225221,MS116,NaN,False,225221,16,False,...,Marcos Giron,[d36bef3a-e455-4180-bc58-b012c8c0b14a],Daniil Medvedev,675d28ec-b177-46b2-959c-595f3ca862a0,12,Night session,1,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,2023-01-28,01:11,https://ausopen.com/match/2023-diede-de-groot-...,235986,DS401,NaN,False,235986,1,False,...,Diede De Groot,[811f507d-277b-4b06-8d10-53e0b88133a5],Yui Kamiji,126de82b-1b67-47a5-b85d-75fea6e4c8b3,2266,Day session,0,NaN,NaN,0
658,2023-01-28,03:37,https://ausopen.com/match/2023-niels-vink-vs-s...,235891,US301,NaN,False,235891,1,False,...,Niels Vink,[bcb78979-3a39-4bb7-b6d3-7a5a35d9525f],Sam Schroder,ca1aafcb-34ac-4023-87cf-88cc84b654ff,2266,Day session,0,NaN,NaN,1
659,2023-01-28,05:39,https://ausopen.com/match/2023-alfie-hewett-vs...,235776,CS401,NaN,False,235776,1,False,...,Alfie Hewett,[66228076-69ee-401e-824e-43e73080fe31],Tokito Oda,8eac5a43-0065-4842-8550-914d5cfb1898,2266,Day session,0,NaN,NaN,2
660,2023-01-29,04:13,https://ausopen.com/match/2023-barbora-krejcik...,229351,WD601,NaN,False,229351,1,True,...,"Barbora Krejcikova, Katerina Siniakova","[b5a8735f-d6f0-48ea-9414-dc5fe5b9c845, 4ba548e...","Shuko Aoyama, Ena Shibahara",49831612-a137-40a3-a491-b51354ed4f2a,12,Twilight session,2,NaN,NaN,0


In [66]:
df_results.to_csv("data/AO_results-all_processed_2023.csv", index=False)

In [73]:
import datetime

In [129]:
t=1673664146765

In [144]:
import json
import base64
import cryptography.hazmat.backends
import cryptography.hazmat.primitives.ciphers
import cryptography.hazmat.primitives.ciphers.algorithms
import cryptography.hazmat.primitives.ciphers.modes
import cryptography.hazmat.primitives.padding

In [145]:
data = {"lastModified":1663265556422,"response":"hlXzkPyyhwUYql2Nwl/3AAcRSsZHKf5LyqsAHqSWjP+ZHzfdmQ7bG2cOrf3YxwcZFIlsJNLJOSL/dSj/fFtjWHkeQd21inSUPOkbu2hSD2xMxEkyss8rOIVJAx6NmY9sap852VtmTc2CT4TdXXRduEK4fXASReIX3Eb9V+TMs24t5ow6w8aau+GWZLP9b32ALs4IZeea+dE3YcKtYrZOu/bV7ZLSawlontkgGN9s4QSjUhv43ifxkS6oDHGFkh+4pjjqfLDa2c0fA28otRZUF4uz+UvYAW2b9hZxBVJQU0E45Bf/myuQjZ14KtQr0NdxAMq53PZlki2hRVtnCDErA2e26cK9/bkC6Pz/J0N7rosTYw6TtDRGPYeqM3z645Uew3f3vEcSQLkWWxi1txQPxTbn1MT4HzRtnAbGJOF+GeaAKbwtSt2B86iHjkyEJ+ssmIMsARRjUmhdFmsMF6vuqA5pSgxvYTacg/yzZvy6HVhZBqTpPcaRJGt41efib3zQg8u++yKXdz8MnHicuz32w/osWzcMsC3Cwm5/a1tJZ48xFJdu8YgUsFS6ioNaO9V6vWz8imQZiPEZxd1FLfRynjS8LpvY3+83M2h+A0oExmcd4UaEMCqkklM1A7ssOXeDTqKS8UiZVM3zH6lzNI42QOZE+WYcPvwNzVLanJpZcKqlLupGfOiHuUclEwKrBL8h3wHtU6UmU+VoPJQM82b4pv5vJY/qlUgjLnaWk18A5UV9MF2b81iI3T8i4U8KGeovMhVLdq7YRZFdBG9djQgPRzwfofB/LRz5+aTwKwiTTsmvy4DMP/2iCB7Eiqr7OaKtuaj1n6vt2MdIstqTz/nDEkjLcdrspajdqHnTfUYLEVJvns6KPIKQaQ61I71G7vkEG4MtZ3PRgGy7/zR/B2qAzhaJmHYMZtOfE2OPcPXi3wi9tTYObYaGzpQIqkFGUtpa862bq8qMSXVUpfb8dvDTOyuvURD9FmSHeDHiO6DYhqxqQrfw1aRHK0vu6QcSsGF31vYnrRGR48nZgouqyzUv90Nc9hvyXBcEaYZpCG2qbAArBseD+RRtXeWV1yvV+C7oy68JOxgLJaL1AsLPX81WV9maPy2Ns3IJ64iNvKMebWFtETNtDPIs5amm+wFjERiQ85DK70wucEd3lWWQr7UddSO8U72whJXGbtsC2onskI75uLF3n7XX4goaHrj0IVB3kVqc4O1zMXWvCzype2EerR2E9K/qoBWh5PQRc4bPhrNdoYGSAh18AKtzVOqPgNgzXnW591r4pWMrWW8Tww89sayPZUnxOwDIaf6kFP74+34K+ZWKGVJA9YBPpKfGAfMgOYalnB7YMA4Tn4Hmt4OQtPeArwgR4DBW+HiQ+aFNK04="}

In [193]:
#data_1 = json.loads("test-court-vision-data.json")
with open("test-court-vision-data.json", 'r') as j:
     contents = json.loads(j.read())

In [276]:
with open("court-vision-nadal-zverev-2022.json", 'r') as j:
     contents_2 = json.loads(j.read())

In [ ]:
def formatDate(t):
    #e = datetime.datetime.now().utcoffset().total_seconds() / 60       # ChatGPT suggestion but not needed
    #t = t + datetime.timedelta(minutes=e)                              # ChatGPT suggestion but not needed
    
    t_tstamp = datetime.datetime.utcfromtimestamp(t/1000)
    n = t_tstamp.day
    r = int(str(n if n >= 10 else "0" + str(n))[::-1])
    i = t_tstamp.year
    a = int(str(i)[::-1])
    o = np.base_repr(int(str(t), base=16), 36).lower() + np.base_repr((i + a) * (n + r), 24).lower()
    s = len(o)
    if s < 14:
        o += "0" * (14 - s)
    elif s > 14:
        o = o[:14]
    return "#" + o + "$"


In [350]:
def decode(data):
    e = formatDate(data['lastModified'])
    n = e.encode()
    r = e.upper().encode()
    cipher = cryptography.hazmat.primitives.ciphers.Cipher(
        cryptography.hazmat.primitives.ciphers.algorithms.AES(n),
        cryptography.hazmat.primitives.ciphers.modes.CBC(r),
        backend=cryptography.hazmat.backends.default_backend()
    )
    decryptor = cipher.decryptor()
    i = decryptor.update(base64.b64decode(data['response'])) + decryptor.finalize()
    unpadder = cryptography.hazmat.primitives.padding.PKCS7(128).unpadder()
    #return json.loads(unpadder.update(i) + unpadder.finalize().decode('utf-8'))
    return json.loads(i.decode("utf-8").replace(i.decode("utf-8")[-1],""))

In [189]:
decode(data)

{'courtId': 1,
 'courtName': None,
 'sessionId': 0,
 'matchStatus': 'C',
 'matchWinner': 1,
 'matchDate': '2022-08-06',
 'playerData': {'tm1Ply1CountryName': 'russia',
  'tm1Ply1Country': 'RUS',
  'tm1Ply1Id': 'MM58',
  'tm1Ply1Name': 'D. MEDVEDEV',
  'tm1Ply1FirstName': 'Daniil',
  'tm1Ply1LastName': 'MEDVEDEV',
  'tm1Ply2CountryName': None,
  'tm1Ply2Country': None,
  'tm1Ply2Id': None,
  'tm1Ply2Name': None,
  'tm1Ply2FirstName': None,
  'tm1Ply2LastName': None,
  'tm1Seed': '1',
  'tm1WinProb': 0.0,
  'tm2Ply1CountryName': 'united-kingdom',
  'tm2Ply1Country': 'GBR',
  'tm2Ply1Id': 'N771',
  'tm2Ply1Name': 'C. NORRIE',
  'tm2Ply1FirstName': 'Cameron',
  'tm2Ply1LastName': 'NORRIE',
  'tm2Ply2CountryName': None,
  'tm2Ply2Country': None,
  'tm2Ply2Id': None,
  'tm2Ply2Name': None,
  'tm2Ply2FirstName': None,
  'tm2Ply2LastName': None,
  'tm2Seed': '3',
  'tm2WinProb': 0.0},
 'matchCenter': {'matchBeats': True,
  'statsPlus': True,
  'strokeSummary': False,
  'courtVision': False,
  

In [259]:
data_json =  decode(contents)

In [289]:
type(data_json)

dict

In [291]:
with open("ATP-WTF_F_Novak-Djokovic-vs-Sebastian-Korda_2022_court-vision.json", 'w') as fp:
    json.dump(data_json, fp)

In [286]:
data_json

{'courtVisionData': [{'a75': True,
   'a76': "Men's Singles",
   'a77': None,
   'a78': 1,
   'a50': {'1_5_3_1': {'a11': 'false',
     'a89': 2.464200022,
     'a12': [{'a70': 11.1929998,
       'a71': 0.86500001,
       'a72': 2.81599998,
       'a73': 'hit'},
      {'a70': 11.1929998, 'a71': 0.86500001, 'a72': 2.81599998, 'a73': 'peak'},
      {'a70': 0.0, 'a71': 0.137999997, 'a72': 0.99000001, 'a73': 'net'},
      {'a70': -4.8210001,
       'a71': -0.208000004,
       'a72': 0.0340000018,
       'a73': 'bounce'},
      {'a70': -16.7549992,
       'a71': -1.01400006,
       'a72': 1.27499998,
       'a73': 'last'}],
     'a90': 'NA',
     'a91': 'Cross Court',
     'a92': 'NA',
     'a81': '1_5_3_1',
     'a13': 'D643',
     'a14': 'D643',
     'a15': 'K0AH',
     'a16': '207.51 KPH',
     'a17': 'NA',
     'a18': 'NA',
     'a93': 1,
     'a94': 1,
     'a19': '0',
     'a20': '3.25 Feet',
     'a21': '207.51 KPH',
     'a22': '0.99 Metre',
     'a23': 'NA',
     'a24': 'NA',
     '

In [252]:
len(data_json['courtVisionData'][0]['a50'].keys())

318

In [351]:
data_json_rg =  decode(contents_2)

In [353]:
with open("RG_SF_Rafael-Nadal-vs-Alexander-Zverev_2022_court-vision.json", 'w') as fp:
    json.dump(data_json_rg, fp)

In [ ]:
########################## 
## Need to make sense of all the "a[something]" key names in the event data json. 

"a50": "pointsData"
'a11': "cruciality"
"a89": 'returnPlacement'
"a12": 'trajectoryData'
'a70': 'x'
'a71': 'y'
'a72': 'z'
'a73': 'position'
'a74': 'erroneousBall'

In [360]:
data_json_rg['courtVisionData'][0].keys()

dict_keys(['a75', 'a76', 'a77', 'a78', 'a50', 'a79', 'a49', 'a80', 'a81', 'a82'])

In [ ]:
'a86': 'uuid' ??
'a85': 'pName'
'a87': 'Country'
'a88': 'seed'

In [386]:
data_json_rg['courtVisionData'][0]['a79']

{'a83': [{'a85': 'R.NADAL', 'a86': '7792', 'a87': 'ESP', 'a88': '5'}],
 'a84': [{'a85': 'A.ZVEREV', 'a86': '30350', 'a87': 'GER', 'a88': '3'}]}

In [ ]:
['isMatchComplete', 'eventType', 'courtName', 'courtId', 'pointsData', 'playersData', 'statsData', 'setsCompleted', 'pointId', 'matchStatus']

In [ ]:
'a90': 'errorType'
'a91': 'winnerPlacement'
'a92': 'unforcedErrorPlacement'
'a81': 'pointId'
'a13': 'serverId'
'a14': 'scorerId'
'a15': 'receiverId'
'a16': 'ballSpeed'
'a17': 'returnSpeed'
'a18': 'returnSpeedFrench'
'a93': 'rallyLength'
'a94': 'rallyLength' + 1 ???
'a19': 'spin'
'a20': 'heightAboveNet'
'a21': 'ballSpeedFrench'
'a22': 'heightAboveNetFrench'
'a23': 'distanceOutsideCourt'
'a24': 'distanceOutsideCourtFrench'
'a95': 'pointEndType'
'a25': 'strokeType'
'a96': 'serveType'
'a97': 'court'
'a98': 'setNumber'
'a99': 'set'
'a100': 'game'
'a101': 'point'
'a102': 'serve'
'a103': 'hand'
'a104': 'breakPoint'
'a26':  'runAroundForeHand'
'a105': 'breakPointConverted'
'a106': 'trappedByNet'


In [ ]:
'a27': 'ballHitCordinate'
'a28': 'ballPeakCordinate'
'a29': 'ballNetCordinate'
'a30': 'ballBounceCordinate'
'a31': 'ballLastCordinate' 
'a32': 'serverCordinate'
'a33': 'receiverCordinate' 
'a34': 'serveBounceCordinate' 

In [ ]:
'a122': 'p1Set1Score',
'a123': 'p1Set2Score',
'a124': 'p1Set3Score',
'a125': 'p1Set4Score',
'a126': 'p1Set5Score',
'a127': 'p1Set1TBScore',
'a128': 'p1Set2TBScore',
'a129': 'p1Set3TBScore',
'a130': 'p1Set4TBScore',
'a131': 'p1Set5TBScore',
'a132': 'p2Set1Score',
'a133': 'p2Set2Score',
'a134': 'p2Set3Score',
'a135': 'p2Set4Score',
'a136': 'p2Set5Score',
'a137': 'p2Set1TBScore',
'a138': 'p2Set2TBScore',
'a139': 'p2Set3TBScore',
'a140': 'p2Set4TBScore',
'a141': 'p2Set5TBScore',
'a142': 'p1GameScore',
'a143': 'p2GameScore'

In [369]:
peter_dict = {'errorType': 'NA',
 'winnerPlacement': 'Cross Court',
 'unforcedErrorPlacement': 'NA',
 'pointId': '1_2_4_1',
 'serverId': '26348',
 'scorerId': '7792',
 'receiverId': '7792',
 'ballSpeed': '182 KPH',
 'returnSpeed': '182 KPH',
 'returnSpeedFrench': '182 KPH',
 'rallyLength': 14,
 'spin': '852.71',
 'heightAboveNet': '5.52 Feet',
 'ballSpeedFrench': '182 KPH',
 'heightAboveNetFrench': '1.68 Metre',
 'distanceOutsideCourt': 'NA',
 'distanceOutsideCourtFrench': 'NA',
 'pointEndType': 'Winner',
 'strokeType': 'NA',
 'serveType': 'Slice',
 'court': 'AdCourt',
 'setNumber': '1',
 'set': '1',
 'game': '2',
 'point': '4',
 'serve': '1',
 'hand': 'BackHand',
 'breakPoint': False,
 'runAroundForeHand': False,
 'breakPointConverted': False,
 'trappedByNet': False,}

In [371]:
len(peter_dict.keys())

31

In [374]:
raw_dict = {'a90': 'NA',
 'a91': 'Cross Court',
 'a92': 'NA',
 'a81': '1_5_2_1',
 'a13': '7792',
 'a14': '7792',
 'a15': '30350',
 'a16': '194 KPH',
 'a17': 'NA',
 'a18': 'NA',
 'a93': 1,
 'a94': 1,
 'a19': '0',
 'a20': '3.66 Feet',
 'a21': '194 KPH',
 'a22': '1.12 Metre',
 'a23': 'NA',
 'a24': 'NA',
 'a95': 'Ace',
 'a25': 'NA',
 'a96': 'Pronated',
 'a97': 'AdCourt',
 'a98': '1',
 'a99': '1',
 'a100': '5',
 'a101': '2',
 'a102': '1',
 'a103': 'ForeHand',
 'a104': False,
 'a26': False,
 'a105': False,
 'a106': False,}

len(raw_dict.keys())

32

In [376]:
point_ids_rg = data_json_rg['courtVisionData'][0]['a50'].keys()

In [377]:
point_ids_rg

dict_keys(['1_5_2_1', '1_9_5_1', '1_13_9_1', '1_8_10_1', '2_1_4_1', '2_3_8_1', '2_7_6_2', '2_9_6_2', '2_10_2_1', '2_10_2_2', '1_1_1_1', '1_1_4_1', '1_1_5_1', '1_1_6_1', '1_3_2_1', '1_3_4_1', '1_3_5_1', '1_3_6_1', '1_5_1_1', '1_5_3_1', '1_5_4_1', '1_5_5_1', '1_7_3_1', '1_7_4_1', '1_9_2_1', '1_9_3_1', '1_9_4_1', '1_9_6_1', '1_11_2_1', '1_11_3_1', '1_11_4_1', '1_11_5_1', '1_11_6_1', '1_11_8_1', '1_11_10_1', '1_13_4_1', '1_13_5_1', '1_13_16_1', '1_13_17_1', '2_2_3_1', '2_2_6_1', '2_2_7_1', '2_2_8_1', '2_4_3_1', '2_6_1_1', '2_6_2_1', '2_6_4_1', '2_6_5_1', '2_6_6_1', '2_6_8_1', '2_8_1_1', '2_8_3_1', '2_8_4_1', '2_8_5_1', '2_8_6_1', '2_10_1_1', '2_10_3_1', '2_10_5_1', '2_12_1_1', '1_8_2_1', '2_1_1_1', '2_7_2_1', '2_7_5_1', '2_8_2_2', '2_10_6_2', '2_11_1_1', '2_12_6_2', '1_1_2_2', '1_2_4_1', '1_2_5_1', '1_4_4_1', '1_4_5_1', '1_7_1_2', '1_7_2_2', '1_8_3_1', '1_8_6_1', '1_8_7_2', '1_8_9_1', '1_10_2_1', '1_10_4_2', '1_10_6_2', '1_10_8_1', '1_10_9_1', '1_10_12_2', '1_10_13_1', '1_10_15_1', '1_10_1

In [379]:
data_json_rg['courtVisionData'][0]['a50']['1_10_8_1']

{'a11': 'true',
 'a89': 3.6677999980000004,
 'a12': [{'a70': 11.2010002,
   'a71': -0.763000011,
   'a72': 3.00999999,
   'a73': 'hit'},
  {'a70': 11.2010002, 'a71': -0.763000011, 'a72': 3.00999999, 'a73': 'peak'},
  {'a70': 0.0, 'a71': 0.416999996, 'a72': 1.14199996, 'a73': 'net'},
  {'a70': -4.85900021,
   'a71': 0.810000002,
   'a72': 0.0390000008,
   'a73': 'bounce'},
  {'a70': -15.118, 'a71': 1.74000001, 'a72': 1.84800005, 'a73': 'peak'},
  {'a70': -18.3059998, 'a71': 1.95899999, 'a72': 1.653, 'a73': 'hit'},
  {'a70': -4.88700008, 'a71': -0.0, 'a72': 4.22100019, 'a73': 'peak'},
  {'a70': -0.0, 'a71': -0.953999996, 'a72': 3.74900007, 'a73': 'net'},
  {'a70': 8.56900024,
   'a71': -2.96099997,
   'a72': 0.0329999998,
   'a73': 'bounce'},
  {'a70': 11.7069998, 'a71': -3.37899995, 'a72': 1.81700003, 'a73': 'peak'},
  {'a70': 11.7069998, 'a71': -3.37899995, 'a72': 1.81700003, 'a73': 'hit'},
  {'a70': 7.20499992, 'a71': -1.96500003, 'a72': 1.94599998, 'a73': 'peak'},
  {'a70': 0.0, 'a71

In [400]:
data_json['courtVisionData'][0]['a50']['3_10_8_2']

{'a11': 'true',
 'a89': 3.98779993,
 'a12': [{'a70': -11.5190001,
   'a71': 1.48899996,
   'a72': 2.9230001,
   'a73': 'hit'},
  {'a70': -11.5190001, 'a71': 1.48899996, 'a72': 2.9230001, 'a73': 'peak'},
  {'a70': 0.0, 'a71': -2.00099993, 'a72': 1.43599999, 'a73': 'net'},
  {'a70': 4.84700012,
   'a71': -3.11899996,
   'a72': 0.0359999985,
   'a73': 'bounce'},
  {'a70': 10.4230003, 'a71': -4.82700014, 'a72': 1.727, 'a73': 'peak'},
  {'a70': 10.4230003, 'a71': -4.82700014, 'a72': 1.727, 'a73': 'hit'},
  {'a70': 6.91300011, 'a71': -3.44700003, 'a72': 1.81099999, 'a73': 'peak'},
  {'a70': 0.0, 'a71': -0.805000007, 'a72': 1.43799996, 'a73': 'net'},
  {'a70': -7.57299995,
   'a71': 1.98500001,
   'a72': 0.0320000015,
   'a73': 'bounce'},
  {'a70': -12.0270004, 'a71': 3.48600006, 'a72': 0.848999977, 'a73': 'peak'},
  {'a70': -12.0270004, 'a71': 3.48600006, 'a72': 0.848999977, 'a73': 'hit'},
  {'a70': -3.10400009, 'a71': 1.26199996, 'a72': 1.55400002, 'a73': 'peak'},
  {'a70': -0.0, 'a71': 0.5

In [ ]:
# Roland Garros Match JSON URL
# https://itp-rg-sls.infosys-platforms.com/prod/api/court-vision/year/2022/eventId/520/matchId/SM001/pointId/0_0_0

# ATP
# https://itp-atp-sls.infosys-platforms.com/static/prod/court-vision/2023/2843/MS001/data.json

# AO
# https://itp-ao-sls.infosys-platforms.com/static/prod/court-vision/2023/580/WS302/data.json


In [404]:
link = f'https://itp-rg-sls.infosys-platforms.com/prod/api/court-vision/year/2022/eventId/520/matchId/SM004/pointId/0_0_0'
# Get request and content from the given link and parse into HTML
pageTree = requests.get(link, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser') 

results_json = json.loads(str(pageSoup))

In [406]:
with open("RG_QF_Novak-Djokovic-vs- Rafael-Nadal_2022_SM004_court-vision.json", 'w') as fp:
    json.dump(decode(results_json), fp)

In [405]:
decode(results_json)['courtVisionData'][0]['a79']

{'a83': [{'a85': 'N.DJOKOVIC', 'a86': '9801', 'a87': 'SRB', 'a88': '1'}],
 'a84': [{'a85': 'R.NADAL', 'a86': '7792', 'a87': 'ESP', 'a88': '5'}]}